In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
%matplotlib inline

In [2]:
def age_10split(x):
    t = int(x['Age'])
    if (t>=0) and (t<10):
        return 0
    elif (t>=10) and (t<20):
        return 1
    elif (t>=20) and (t<30):
        return 2
    elif (t>=30) and (t<40):
        return 3
    elif (t>=40) and (t<50):
        return 4
    elif (t>=50) and (t<60):
        return 5
    elif (t>=60) and (t<70):
        return 6
    elif (t>=70):
        return 7
    
def name_split(x):
    ret = 'Unknown'
    t = str(x['Name'])
    sex = str(x['Sex'])
    t = t.split()
    for i in t:
        if(sex == 'male'):
            if (i == 'Master.'):
                return i
            else:
                ret = 'Mr.'
        elif(sex == 'female'):
            if(i == 'Mrs.'):
                return i
            else:
                ret = 'Miss.'
        else:
            ret = 'Unknown'
    return ret

def Cabin_split(x):
    t = x['Cabin']
    c = str(t[0])
    if c == 'A' or c == 'a':
        return 0
    elif c == 'B' or c == 'b':
        return 1
    elif c == 'C' or c == 'c':
        return 2 
    elif c == 'D' or c == 'd':
        return 3 
    elif c == 'E' or c == 'e':
        return 4 
    elif c == 'F' or c == 'f':
        return 5
    elif c == 'G' or c == 'g':
        return 6
    elif c == 'H' or c == 'h':
        return 7
    elif c == 'I' or c == 'i':
        return 8
    else:
        return 9
    
def ticket_split(x,DF):
    s = str(x['Ticket'])
    n = (len(DF.Ticket.loc[DF.Ticket == s].index.tolist()))
    if n == 1:
        return 1
    elif n == 2:
        return 2
    elif n == 3:
        return 3
    elif n == 4:
        return 4
    elif n == 5:
        return 5
    elif n == 6:
        return 6
    elif n > 6:
        return 7

def Fare_rounddown(x):
    s = int(x['Fare'])
    return s

def Fare_split(x):
    s = int(x['Fare_Rounddown'])
    if (s >= 0) and (s < 10):
        return 0
    elif (s >= 10) and (s < 20):
        return 1
    elif (s >= 20) and (s < 30):
        return 2
    elif (s >= 30) and (s < 40):
        return 3
    elif (s >= 40) and (s < 50):
        return 4
    elif (s >= 50) and (s < 60):
        return 5
    elif (s >= 60) and (s < 70):
        return 6
    elif (s >= 70) and (s < 80):
        return 7
    elif (s >= 80) and (s < 90):
        return 8
    elif (s >= 90) and (s < 100):
        return 9
    elif (s >= 100) and (s < 110):
        return 10
    elif (s >= 110) and (s < 120):
        return 11
    elif (s >= 120) and (s < 130):
        return 12
    elif (s >= 130):
        return 13
    

def concat_SibSp_Parch(x):
    t1 = int(x['SibSp'])
    t2 = int(x['Parch'])
    if (t1+t2+1 > 7):
        return 7
    return t1+t2+1
    
def is_Alone(x):
    t = int(x['SibParch'])
    if t == 1:
        return 1
    return 0

def has_Cabin(x):
    t = str(x['Cabin'])
    if t == 'Unknown':
        return 0
    else:
        return 1

titanic_feature_observationにて新たに作成したcsvファイルを読み込む     
ここではAgeの欠損値を中央値で補完してSurvivedの識別モデルを構築する      

In [3]:
# CSVファイルを読み込む
train_df = pd.read_csv('new_titanic_train.csv')
test_df = pd.read_csv('new_titanic_test.csv')

# Unnamed が邪魔なので削除する
drop_columns = ['Unnamed: 0']
train_df = train_df.drop(drop_columns, axis=1)
test_df = test_df.drop(drop_columns, axis=1)

Ageの欠損値を各データセットにおける中央値で置換する

In [4]:
train_df['Age'] = train_df[['Age']].fillna(test_df[['Age']].median())
test_df['Age'] = test_df[['Age']].fillna(test_df[['Age']].median())

trainとtestにおけるAgeの欠損値が無くなったか確認する

In [5]:
train_df['Age'].isnull().sum()

0

In [6]:
test_df['Age'].isnull().sum()

0

年齢を10歳ごとにカテゴリ分けしたBy_Age_Classという属性を新たに作成する

In [7]:
train_df['By_Age_Class'] = train_df.apply(lambda x: age_10split(x), axis=1)
test_df['By_Age_Class'] = test_df.apply(lambda x: age_10split(x), axis=1)

In [8]:
train_df.isnull().sum()

PassengerId             0
Survived                0
Pclass                  0
Name                    0
title_name              0
onehot_title            0
Sex                     0
onehot_sex              0
Age                     0
SibSp                   0
Parch                   0
SibParch                0
Ticket                  0
Ticket_num              0
Fare                    0
Fare_Rounddown          0
Fare_Rounddown_Split    0
Cabin                   0
isCabin                 0
Embarked                0
onehot_Embarked         0
By_Age_Class            0
dtype: int64

In [9]:
test_df.isnull().sum()

PassengerId             0
Pclass                  0
Name                    0
title_name              0
onehot_title            0
Sex                     0
onehot_sex              0
Age                     0
SibSp                   0
Parch                   0
SibParch                0
Ticket                  0
Ticket_num              0
Fare                    0
Fare_Rounddown          0
Fare_Rounddown_Split    0
Cabin                   0
isCabin                 0
Embarked                0
onehot_Embarked         0
By_Age_Class            0
dtype: int64

生存者の予測に必要なデータの穴埋めが完了したので生存者の予測モデルを作成する
予測に必要のないカラムを消去する

In [10]:
train_drop_columns = ['PassengerId','Name','title_name','Sex','Age','SibParch','Ticket','Fare_Rounddown','Fare','Cabin','Embarked']
train_df2 = train_df.drop(train_drop_columns, axis=1)

In [11]:
train_df2.head(2)

,Survived,Pclass,onehot_title,onehot_sex,SibSp,Parch,Ticket_num,Fare_Rounddown_Split,isCabin,onehot_Embarked,By_Age_Class
0,0,3,2,1,1,0,1,0,0,2,2
1,1,1,3,0,1,0,1,7,1,0,3


In [12]:
# データを訓練データ、テストデータに分割する
from sklearn.model_selection import train_test_split

train, test = train_test_split(train_df2, test_size = 0.3)
# indexをリセットする
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
print(len(train),len(test))

623 268


In [13]:
# 特徴として使うデータとラベルとして使うデータに分ける
X_train = np.array(train.drop('Survived', axis=1).values).astype(np.float32)
y_train = np.array(train['Survived'].values).astype(np.float32)

X_test = np.array(test.drop('Survived', axis=1).values).astype(np.float32)
y_test = np.array(test['Survived'].values).astype(np.float32)

print(X_train.shape,y_train.shape)

(623, 10) (623,)


生存者の予測モデルを構築する     
Tensorflowを用いたNNのロジスティック回帰とXGboostで各モデルの精度を比較する

In [14]:
# 入力層
X = tf.placeholder(tf.float32, shape=[None, 10], name="input")
t = tf.placeholder(tf.float32, shape=[None, 1])
# パラメータ1
stddev = np.sqrt(2.0 / 10)
W1 = tf.Variable(tf.truncated_normal([10,24], stddev=stddev))
b1 = tf.Variable(tf.constant(0.1, shape=[24]))

# パラメータ2
stddev = np.sqrt(2.0 / 24)
W2 = tf.Variable(tf.truncated_normal([24,48], stddev=stddev))
b2 = tf.Variable(tf.constant(0.1, shape=[48]))
keep_prob = tf.placeholder(tf.float32) # ドロップアウトする割合

# パラメータ3
stddev = np.sqrt(2.0 / 48)
W3 = tf.Variable(tf.truncated_normal([48,1], stddev=stddev))
b3 = tf.Variable(tf.constant(0.1, shape=[1]))

layer1 = tf.nn.relu(tf.matmul(X,W1) + b1)

layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
layer2_drop = tf.nn.dropout(layer2, keep_prob)

layer3 = tf.matmul(layer2_drop,W3)+b3

p = tf.nn.sigmoid(layer3,name="output")
# 荷重減衰
norm_term = tf.nn.l2_loss(layer1) + tf.nn.l2_loss(layer2_drop)
# 正則項
lambda_ = 0.001
# 損失関数
loss = tf.reduce_mean(tf.square(p - t))+ lambda_*norm_term
# 学習アルゴリズム
optimizer = tf.train.AdamOptimizer()
train_step = optimizer.minimize(loss)
# 精度
correct_prediction = tf.equal(tf.sign(p-0.5), tf.sign(t-0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 30
for i in range(30000):
    batch_mask = np.random.choice(X_train.shape[0], batch_size)
    x_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]
    sess.run(train_step,feed_dict={X: x_batch, t: y_batch.reshape(-1,1), keep_prob:0.5})
    if i % 3000 == 0:
        train_acc, train_loss = sess.run([accuracy,loss], feed_dict={X: X_train, t: y_train.reshape(-1,1), keep_prob:1.0})
        test_acc, test_loss = sess.run([accuracy,loss], feed_dict={X: X_test, t: y_test.reshape(-1,1), keep_prob:1.0})
        print("[Train] step: %d, loss: %f, acc: %f, [Test] loss: %f, acc : %f" % (i, train_loss, train_acc,test_loss,test_acc))

[Train] step: 0, loss: 61.435287, acc: 0.382022, [Test] loss: 28.931728, acc : 0.388060
[Train] step: 3000, loss: 0.327592, acc: 0.865169, [Test] loss: 0.265813, acc : 0.805970
[Train] step: 6000, loss: 0.243979, acc: 0.895666, [Test] loss: 0.234455, acc : 0.798507
[Train] step: 9000, loss: 0.202054, acc: 0.890851, [Test] loss: 0.219193, acc : 0.802239
[Train] step: 12000, loss: 0.183529, acc: 0.892456, [Test] loss: 0.212868, acc : 0.813433
[Train] step: 15000, loss: 0.180292, acc: 0.890851, [Test] loss: 0.217516, acc : 0.798507
[Train] step: 18000, loss: 0.168366, acc: 0.890851, [Test] loss: 0.212061, acc : 0.791045
[Train] step: 21000, loss: 0.159919, acc: 0.895666, [Test] loss: 0.222104, acc : 0.768657
[Train] step: 24000, loss: 0.157481, acc: 0.903692, [Test] loss: 0.215337, acc : 0.787313
[Train] step: 27000, loss: 0.166086, acc: 0.887640, [Test] loss: 0.224480, acc : 0.772388


In [16]:
from xgboost import XGBRegressor
xg_clf = XGBRegressor(max_delta_step=0)

xg_model = xg_clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


log_lossを用いて各モデルの誤差で比較を行う

In [17]:
from sklearn.metrics import log_loss
nn_pred = sess.run(p,feed_dict={X:X_test,keep_prob:1.0}).reshape(-1,)
xg_pred = xg_clf.predict(X_test)
nn_pred = np.fmin(0.999,nn_pred)
nn_pred = np.fmax(0.0001,nn_pred)
xg_pred = np.fmin(0.999,xg_pred)
xg_pred = np.fmax(0.0001,xg_pred)
print('NN LogisticRegression log loss = {}'.format(log_loss(y_test,nn_pred)))
print('XGBoost log loss = {}'.format(log_loss(y_test,xg_pred)))

NN LogisticRegression log loss = 0.6700371422161133
XGBoost log loss = 0.3766944857569756


作成したモデルをテストデータに適用する

In [18]:
test_drop_columns = ['PassengerId','Name','title_name','Sex','Age','SibParch','Ticket','Fare_Rounddown','Fare','Cabin','Embarked']
test_df_pred = test_df.drop(test_drop_columns, axis=1)

In [19]:
# 訓練データと同じカラムがあるか確認する
test_df_pred.head(2)

,Pclass,onehot_title,onehot_sex,SibSp,Parch,Ticket_num,Fare_Rounddown_Split,isCabin,onehot_Embarked,By_Age_Class
0,3,2,1,0,0,1,0,0,1,3
1,3,3,0,1,0,1,0,0,2,4


In [21]:
X_pred = np.array(test_df_pred.values.astype(np.float32))

XGboostかロジスティック回帰かどちらかのモデルで予測をする

In [23]:
#test_pred = xg_clf.predict(X_pred)
test_pred = sess.run(p,feed_dict={X:X_pred,keep_prob:1.0})

In [24]:
# 予測結果を少し確認する
test_pred[:4]

array([[  2.09809661e-01],
       [  5.93535416e-02],
       [  6.88267755e-05],
       [  7.97101706e-02]], dtype=float32)

提出ファイルに予測結果を書き込む

In [25]:
submit_df = pd.read_csv('gender_submission.csv')
submit_df.head(10)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [26]:
for i in range(0,test_pred.shape[0]):
    if test_pred[i] < 0.5:
        submit_df['Survived'].iloc[i] = 0
    else:
        submit_df['Survived'].iloc[i] = 1

提出ファイルを別名で保存する

In [41]:
submit_df.to_csv('titanic_submission.csv',index=False)

元の提出ファイルと予測値が変わっているか確認する

In [27]:
show = pd.read_csv('titanic_submission.csv')
show.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,1
8,900,1
9,901,0
